In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_meta = pd.read_csv("/kaggle/input/birdclef-2022/train_metadata.csv")
labels = list(train_meta['primary_label'].unique()) #grabs the labels

# Imports

In [ ]:
import os
import json 
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.utils import plot_model, Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Concatenate
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

# **Preprocessing/Data Preparation**

In [ ]:
import soundfile as sf
import os

def cutAudio(file_path, is_save):
    # Load the file
    filename = file_path.replace("/", "_")
    file_path = "/kaggle/input/birdclef-2022/train_audio/" + file_path
    audio, sr = librosa.load(file_path)

    # Get number of samples for 5 seconds
    buffer = 5 * sr
    totalSamples = len(audio)
    samplesProcessed = 0
    count = 1

    audioSplit = []
    audioFilenames = []
    while samplesProcessed < totalSamples:
        #check if the buffer is not exceeding total samples 
        if buffer > (totalSamples - samplesProcessed):
            buffer = totalSamples - samplesProcessed

        block = audio[samplesProcessed : (samplesProcessed + buffer)]
        audioSplit.append(block)

        # Write 5 second 
        if is_save == True:
            outputFilename = "/kaggle/working/each5s/split_" + str(count) + "_" + filename
            audioFilenames.append(outputFilename)
            sf.write(outputFilename, block, sr)
        count += 1
        samplesProcessed += buffer
    return audioSplit, sr, audioFilenames

In [ ]:
def splitTrainAudio(_df):
    data = []
    #iterate through the rows
    for index, row in tqdm(_df.iterrows()):
        cutAudio(row["filename"], True)
        audio_lst, sr, filenames = cutAudio(row["filename"], True)
        #loop through audios
        for idx, y in enumerate(audio_lst):
            data.append([row["primary_label"], row["filename"], filenames[idx]])

    data_df = pd.DataFrame(data, columns=['primary_label', 'original_filename', 'filename'])
    data_df.to_csv("/kaggle/working/data_df.csv", index=False)

In [ ]:
data_frames = []
for label in labels:
    tmp_df = train_meta[train_meta["primary_label"] == label].sample(n=1, replace=True).reset_index(drop=True)
    data_frames.append(tmp_df)
sample_df = pd.concat(data_frames).reset_index(drop=True)
sample_df

In [ ]:
#make a directory for the samples
!mkdir -p "/kaggle/working/each5s"
splitTrainAudio(sample_df)

In [ ]:
data_df = pd.read_csv("/kaggle/working/data_df.csv")
data_df

In [ ]:
num_rows = 216
num_columns = 216
num_channels = 1
n_mels = 512

#grab features using mfcc
def extractFeatures(y, sr):
    feat = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_rows, n_mels=n_mels)
    if feat.shape[1] <= num_columns:
        pad_width = num_columns - feat.shape[1]
        feat = np.pad(feat, pad_width=((0,0),(0,pad_width)), mode='constant')
    return feat

In [ ]:
#data generator for training and testing data
class DataGenerator(Sequence):
    #initial config
    def __init__(self,
                _X,
                batch_size=32,
                n_channels=1,
                n_columns=470,
                n_rows=120,
                shuffle=True):
        self.batch_size = batch_size
        self.X = _X
        self.n_channels = n_channels
        self.n_columns = n_columns
        self.n_rows = n_rows
        self.shuffle = shuffle
        self.img_indexes = np.arange(len(self.X))
        self.on_epoch_end()
    
    #returns the number of batches per epoch
    def __len__(self):
        return int(np.floor(len(self.img_indexes) / self.batch_size))
    
    #generates a batch of data
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temps = [self.img_indexes[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temps)
        return X, y
    
    #defines what happens after epoch, which is to update the indexes
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    #data generation function which returns the data and the labels and encodes them using keras to_categorical
    def __data_generation(self, list_IDs_temps):
        X = np.empty((self.batch_size, self.n_rows, self.n_columns))
        y = np.empty((self.batch_size), dtype=int)
        for i, ID in enumerate(list_IDs_temps):
            file_path = self.X.iloc[ID]["filename"]
            audio, sr = librosa.load(file_path)
            feat = extractFeatures(audio, sr)
            x_features = feat.tolist()
            label = self.X.iloc[ID]["target"]
            X[i] = np.array(x_features)
            y[i] = label
        X = X.reshape(X.shape[0], self.n_rows, self.n_columns, self.n_channels)
        
        return X, to_categorical(y, num_classes=len(labels))

In [ ]:
#more config
params = dict(
    batch_size=128,
    n_rows=num_rows,
    n_columns=num_columns,
    n_channels=num_channels,
)
params_train = dict(
    shuffle=True,
    **params
)
params_valid = dict(
    shuffle=False,
    **params
)

In [ ]:
import matplotlib.pyplot as plt

#plotting function to allow me to view what is happening with my model
def plot_his(history):
    plt.figure(1, figsize = (15,8))
    plt.subplot(221)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'])
    plt.subplot(222)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'])
    plt.show()

In [ ]:
#Convolutional Neural Network architecture 
def create_cnn():
    base_model = Sequential()
    base_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(num_rows, num_columns, num_channels)))
    base_model.add(MaxPooling2D((2, 2)))
    base_model.add(Dropout(0.2))
    base_model.add(Conv2D(64, (3, 3), activation='relu'))
    base_model.add(MaxPooling2D((2, 2)))
    base_model.add(Dropout(0.2))
    base_model.add(Conv2D(64, (3, 3), activation='relu'))
    base_model.add(Flatten())
    base_model.add(Dense(len(labels), activation='softmax', kernel_regularizer='l1'))
    return base_model

In [ ]:
data_df['target'] = data_df['primary_label'].apply(lambda x: labels.index(x))
data_df

In [ ]:
from keras import backend as K

#equations for recall, precision and f1
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
#function for training the model
def train_model(model, train_gen, val_gen):
    checkpoint_model_path = "/kaggle/working/customModel.h5"
    metric = "val_accuracy"
    #compiling model with metrics as accuracy, f1, precision and recall to allow us to see what is happening per epoch
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
    num_epochs = 50
    
    #added a checkpointer to allow saving of weights at an interval, saving only when it achieves better scores
    checkpointer = ModelCheckpoint(
        filepath=checkpoint_model_path,
        monitor=metric, verbose=1, save_best_only=True)
    
    #this stops the model from training when the metric being monitored is no longer improving
    es_callback = EarlyStopping(monitor=metric, patience=5, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor=metric, factor=0.3, patience=1, verbose=1, min_delta=0.0001, cooldown=1, min_lr=0.00001)
    
    #fit the model
    history = model.fit(
        train_gen,
        epochs=num_epochs,
        validation_data=val_gen,
        callbacks=[checkpointer,es_callback,reduce_lr],
        verbose=1
    )
    
    #evaluate the model
    loss, accuracy, f1_score, precision, recall = model.evaluate(val_gen, verbose=0)
    
    #print all the scores
    print("Loss :" + str(loss))
    print("Accuracy :" + str(accuracy))
    print("F1 :" + str(f1_score))
    print("Precision :" + str(precision))
    print("Recall :" + str(recall))
    
    #plot the model
    plot_his(history)
    
    
    return model

In [ ]:
#using an 90:10 split between training:testing
X_train, X_valid, _, _ = train_test_split(data_df, data_df["target"], test_size=0.1, random_state=42)
train_generator = DataGenerator(X_train, **params_train)
valid_generator = DataGenerator(X_valid, **params_valid)
cnn_model = create_cnn()
cnn_model = train_model(cnn_model, train_generator, valid_generator)

In [ ]:
#grab all the unique labels
labels = list(data_df['primary_label'].unique())

# **Submission**

In [ ]:
test_path = "/kaggle/input/birdclef-2022/test_soundscapes/"
files = [f.split('.')[0] for f in sorted(os.listdir(test_path))]

birds_path = "/kaggle/input/birdclef-2022/scored_birds.json"
with open(birds_path) as bf:
    birds = json.load(bf)

data = []
for f in files:
    file_path = test_path + f + '.ogg'
    audio, sr = librosa.load(file_path)
    # Get number of samples for 5 seconds; replace 5 by any number
    buffer = 5 * sr
    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    while samples_wrote < samples_total:
        #check if the buffer is not exceeding total samples 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote : (samples_wrote + buffer)]
        feat = extractFeatures(block, sr)
        x = feat.reshape(1, num_rows, num_columns, num_channels)
        pred = cnn_model.predict(x)
        label_index = np.argmax(pred,axis=1)[0]
        
        for b in birds:
            segment_end = counter * 5   
            row_id = f + '_' + b + '_' + str(segment_end)
            target = False
            if labels[label_index] == b:
                target = True
            data.append([row_id, target])
        counter += 1
        samples_wrote += buffer
        
submission_df = pd.DataFrame(data, columns=['row_id', 'target'])
print(submission_df.head())

In [ ]:
#removing the samples folder as it messes up finding the submission.csv
import shutil
shutil.rmtree(os.getcwd() + '/each5s')

In [ ]:
#saving on /kaggle/ and /kaggle/working to mitigate against the problem where the submission.csv cannot be found
submission_df.to_csv("/kaggle/submission.csv", index=False)
submission_df.to_csv("/kaggle/working/submission.csv", index=False)